In [ ]:
import torch
import os
from itertools import product
from random import shuffle
from tqdm import tqdm

from KNN_Embeddings import *

In [ ]:
# Feature normalization
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from torch.utils.data import DataLoader, TensorDataset, Dataset

# Create a scaler object
scaler = StandardScaler()

# Fit on training data and transform both training and test data
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)

X_train_tensor = torch.tensor(X_train_normalized, dtype=torch.float)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test_normalized, dtype=torch.float)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Creating DataLoader instances
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

class EmbeddingDataset(Dataset):
    def __init__(self, embeddings, labels):
        self.embeddings = embeddings
        self.labels = labels
    
    def __len__(self):
        return len(self.embeddings)
    
    def __getitem__(self, index):
        # Return embedding, label, and index
        return self.embeddings[index], self.labels[index], index

train_dataset = EmbeddingDataset(X_train_tensor, y_train_tensor)
test_dataset = EmbeddingDataset(X_test_tensor, y_test_tensor)        

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)

## DPSH Model 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

import torch
import torch.nn as nn
import torch.nn.functional as F

class DPSH(nn.Module):
    def __init__(self, input_dim, num_bits):
        super(DPSH, self).__init__()
        self.fc1 = nn.Linear(input_dim, 512)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(512, num_bits)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = torch.tanh(x)  # Ensuring outputs are bounded between -1 and 1
        x = F.normalize(x, p=2, dim=1)  # L2 normalization
        return x

def to_one_hot(labels, num_classes):
    """ Convert labels to one-hot encoded format """
    return torch.eye(num_classes)[labels].to(labels.device).squeeze(1)

# Modify the DPSHLoss class to convert labels to one-hot
class DPSHLoss(torch.nn.Module):
    def __init__(self, config):
        super(DPSHLoss, self).__init__()
        self.config = config
        self.U = torch.zeros(config["num_train"], config["bit"]).float().to(config["device"])
        self.Y = torch.zeros(config["num_train"], config["n_class"]).float().to(config["device"])
    
    def forward(self, u, y, ind):
        y_one_hot = to_one_hot(y, self.config["n_class"])  # Convert labels to one-hot
        self.U[ind, :] = u.data
        self.Y[ind, :] = y_one_hot.data
        
        s = (y_one_hot @ self.Y.t() > 0).float()
        inner_product = u @ self.U.t() * 0.5
        likelihood_loss = (1 + (-(inner_product.abs())).exp()).log() + inner_product.clamp(min=0) - s * inner_product
        likelihood_loss = likelihood_loss.mean()
        quantization_loss = self.config["alpha"] * (u - u.sign()).pow(2).mean()

        return likelihood_loss + quantization_loss

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

config = {
    "num_train": len(train_dataset),  # Total number of training samples
    "n_class": 9,     # Total number of classes
    "device": device,                 # Device to run the model (e.g., 'cuda' or 'cpu')
    "alpha": 0.1,                    # Alpha parameter for quantization loss
    "bit": 12
}

model = DPSH(input_dim=X_train.shape[1], num_bits=12).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
dpsh_loss = DPSHLoss(config)

## Train DPSH Model

In [ ]:


def calculate_accuracy(outputs, labels):
    # Get the predicted labels by finding the nearest neighbor in the embedding space
    with torch.no_grad():
        distances = torch.cdist(outputs, outputs)
        nearest_neighbors = distances.argsort(dim=1)[:, 1]  # The nearest non-identical item
        predicted_classes = labels[nearest_neighbors]
        correct_predictions = (predicted_classes == labels).float()
        accuracy = correct_predictions.sum() / len(labels)
    return accuracy


num_epochs = 50
for epoch in range(num_epochs):
    running_loss = 0.0
    running_accuracy = 0.0
    for data, target, index in train_loader:
        data, target, index = data.to(device), target.to(device), index.to(device)
        optimizer.zero_grad()
        
        outputs = model(data)
        loss = dpsh_loss(outputs, target, index)
        accuracy = calculate_accuracy(outputs, target)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        running_accuracy += accuracy.item()
    
    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = running_accuracy / len(train_loader)
    print(f'Epoch {epoch+1}: Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}')

## KNN on hashed embeddings

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import label_binarize
import numpy as np

def get_binary_hash_codes(model, loader, device):
    model.eval()
    hash_codes = []
    labels = []
    with torch.no_grad():
        for data, target, _ in loader:  # Added '_' to handle the 'index' being returned
            data = data.to(device)
            outputs = model(data)
            binary_codes = torch.sign(outputs).cpu().numpy()  # Convert to binary hash codes
            hash_codes.extend(binary_codes)
            labels.extend(target.cpu().numpy())
    return np.array(hash_codes), np.array(labels)

# Ensure model and device are defined and properly initialized
# Example: model = DPSH(input_dim=X_train.shape[1], num_bits=48).to(device)
# and device is defined like device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Extract hash codes
train_codes, train_labels = get_binary_hash_codes(model, train_loader, device)
test_codes, test_labels = get_binary_hash_codes(model, test_loader, device)

# Classification with KNN
knn = KNeighborsClassifier(n_neighbors=5, metric='hamming')
knn.fit(train_codes, train_labels)
predictions = knn.predict(test_codes)
y_pred_proba = knn.predict_proba(test_codes)

print(classification_report(test_labels, predictions))

# Binarize the labels for a one-vs-rest computation
y_test_binarized = label_binarize(test_labels, classes=np.unique(train_labels))  # Updated to use `test_labels`

# Calculate the average precision for each class
average_precisions = []
for i in range(y_test_binarized.shape[1]):  # iterate over classes
    average_precisions.append(average_precision_score(y_test_binarized[:, i], y_pred_proba[:, i]))

# Compute the mean of the average precisions
map_score = np.mean(average_precisions)
print(f'Mean Average Precision (MAP): {map_score}')
